In [1]:
import nipype

from nipype import Node, Workflow
from nipype.interfaces import fsl
from nipype.interfaces import ants
from nipype.interfaces import freesurfer
from os.path import abspath

In [2]:
base_dir=abspath('examples/')

In [3]:
CORES=4
TASKS=2

In [4]:
# grabber asks you how to take input files,
# sink where/how to save final output file
#everything else is saved in a temp directory

grabber = nipype.Node(interface=nipype.DataGrabber( outfields=['out_file']), name='grabber')                      
grabber.inputs.base_directory = base_dir
grabber.inputs.sort_filelist = True
grabber.inputs.template = '*/t1w.nii.gz' #all folders, all files named t1w.nii.gz

sink = nipype.Node(interface=nipype.DataSink(),name='sink')
sink.inputs.base_directory = base_dir


In [5]:
# Neck removal by FSL robustfov
neck_remove=nipype.MapNode(interface=fsl.RobustFOV(), name='neck_remove', iterfield=['in_file'])
neck_remove.inputs.out_roi="t1w_fov.nii.gz"

# Field Inhomogenity estimation (if any) and removal by ANTs N4BiasFieldCorrection
N4_FC=nipype.MapNode(interface=ants.N4BiasFieldCorrection(),name="N4_FC", iterfield=['input_image'])
N4_FC.inputs.dimension = 3
N4_FC.inputs.output_image="t1w_fov_N4.nii.gz"
N4_FC.inputs.num_threads=CORES

In [6]:
workflow = nipype.Workflow('workflow')
workflow.connect([(grabber, neck_remove, [('out_file', 'in_file')]),
                  (neck_remove, N4_FC, [('out_roi', 'input_image')]),
                  (N4_FC, sink, [('output_image', '@in_file.@final')])])

In [7]:
workflow.run()
#workflow.run('MultiProc', plugin_args={'n_procs': TASKS})

170921-10:57:56,338 workflow INFO:
	 Workflow workflow settings: ['check', 'execution', 'logging']
170921-10:57:56,406 workflow INFO:
	 Running serially.
170921-10:57:56,408 workflow INFO:
	 Executing node grabber in dir: /var/folders/5_/33p0sl3j2_ngqn_gghyy2zj00000gn/T/tmpM4tAw3/workflow/grabber
170921-10:57:56,462 workflow INFO:
	 Executing node neck_remove in dir: /var/folders/5_/33p0sl3j2_ngqn_gghyy2zj00000gn/T/tmp_LuIz3/workflow/neck_remove
170921-10:57:56,470 workflow INFO:
	 Executing node _neck_remove0 in dir: /var/folders/5_/33p0sl3j2_ngqn_gghyy2zj00000gn/T/tmp_LuIz3/workflow/neck_remove/mapflow/_neck_remove0
170921-10:57:56,478 workflow INFO:
	 Running: robustfov -i /Users/poggiali/GitHub/dementiawindow/examples/another_guy/t1w.nii.gz -r t1w_fov.nii.gz -m t1w_to_ROI.nii.gz
170921-10:58:14,528 interface INFO:
	 stdout 2017-09-21T10:58:14.528189:Final FOV is: 
170921-10:58:14,529 interface INFO:
	 stdout 2017-09-21T10:58:14.528189:0.000000 180.000000 0.000000 560.000000 140.000